In [18]:
from Portfolio import Portfolio
import warnings
import numpy as np
from datetime import datetime, timedelta
import ipywidgets as widgets
import IPython
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

vb = widgets.VBox()
portfolio = None
out = widgets.Output(layout = {
    'border': '1px solid black',
})

def simulate_portfolio():
    IPython.display.clear_output(wait=True)
    global portfolio
    tickers = []
    weights = []
    for hb in vb.children[1:-1]:
        tickers.append(hb.children[0].value)
        weights.append(hb.children[1].value)
    sd, ed, inv = [w.value for w in vb.children[0].children]
    portfolio = Portfolio(tickers, weights, inv, sd, ed)
    out.clear_output()
    with out:
        portfolio.summary()
        portfolio.plot()
        plt.show()

def btn_add_ticker(obj):
    remove = vb.children[-1]
    vb.children = vb.children[:-1]
    vb.children=tuple(list(vb.children) + [add_ticker(), remove])
    
def btn_finish_portfolio(obj):
    weights = 0
    for hb in vb.children[1:-1]:
        weights += hb.children[1].value
    try: 
        np.testing.assert_almost_equal(weights, 1)
    except:
        print("Weights must sum up to 1")
        return
    sd, ed, inv = [w.value for w in vb.children[0].children]
    if not datetime.strptime(sd, "%Y-%m-%d") < datetime.strptime(ed, "%Y-%m-%d"):
        print("You must enter a valid date and start date must be earlier than the end date")
        return
    if not inv > 0:
        print("Investment must be positive")
        return
    simulate_portfolio()

def add_ticker():
    w1 = widgets.Text(
        placeholder='URTH',
        description='Ticker:',
        disabled=False
    )

    w2 = widgets.BoundedFloatText(
        placeholder=0,
        min=0,
        max=1,
        step=0.05,
        description='Weight:',
        disabled=False
    )
    
    b_remove = widgets.Button(
        value=False,
        description='Remove Ticker',
        disabled=False,
        button_style='', tooltip='Description'
    )
    
    hbox = widgets.HBox([w1, w2, b_remove])
    
    def btn_remove_ticker(obj):
        children = []
        for hb in list(vb.children):
            if hb != hbox:
                children.append(hb)
            else:
                hb = None
        vb.children = children
    
    b_remove.on_click(btn_remove_ticker)
        
    return hbox


def date_inv_input():
    w_sd = widgets.Text(
            value=(datetime.today() - timedelta(days=365)).strftime("%Y-%m-%d"),
            description='Start Date:',
            disabled=False
            )

    w_ed = widgets.Text(
            value=datetime.now().strftime("%Y-%m-%d"),
            description='End Date:',
            disabled=False
            )

    w_inv = widgets.FloatText(
            value=10000,
            step=1000,
            description='Investment:',
            disabled=False
            )

    return widgets.HBox([w_sd, w_ed, w_inv])

def buttons_input():
    b_add = widgets.Button(
        value=False,
        description='Add Ticker',
        disabled=False,
        button_style=''
    )

    b_simulate = widgets.Button(
        value=False,
        description='Simulate Portfolio',
        disabled=False,
        button_style=''
    )

    b_add.on_click(btn_add_ticker)
    b_simulate.on_click(btn_finish_portfolio)
    return widgets.HBox([b_add, b_simulate])


vb.children=tuple(list(vb.children) + [date_inv_input(), add_ticker(), buttons_input()])
display(vb)
display(out)

Output(layout=Layout(border='1px solid black'))